<a href="https://colab.research.google.com/github/samvatsan/Eutrophication/blob/main/eutrophication_loads_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive
#!pip install -U protobuf==3.8.0

# Hee hee - ANN
!pip install tensorflow-gpu
import tensorflow as tf

# Needed to manipulate data. Data frames help with this
import pandas as pd

# Needed to convert dataframes into numpy arrays to feed keras
import numpy as np

# Needed to access water quality data files from Gdrive 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Keras requirements
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Authenticate and create the PyDrive client. All my data is in there
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Function to read CSV file into a pandas dataframe. We will use this function
# to read in water quality data from various sources into data frames
def create_loads(link, comment_separator='#'):
  #link should be a drive.google.com link of the form: https://drive.google.com/open?id=1-6mD_PTOYuVTPEtTK9pZUK4EUymj7K6V
  # Note the ?id=<blah> - that should be there for this function to parse the link id correctly
  fluff, id = link.split('=')
  print ("Loading: ", id) # Verify that you have everything after '='

  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('someTempFile')  
  loads = pd.read_csv('someTempFile', comment=comment_separator)
  return loads   # This would be a dataframe loaded from the gdrive csv


In [ ]:
# Load aflow, ph, DO, temp, Turbidity as is. We will clean the data once we have
# them in data frames
inrmloads = create_loads('https://drive.google.com/open?id=1-6mD_PTOYuVTPEtTK9pZUK4EUymj7K6V')
phloads = create_loads('https://drive.google.com/open?id=1sp1WCu_aavxN27mxB6LIv5qzm694VUJe')
doloads = create_loads('https://drive.google.com/open?id=1pMuQPazmrlovFSuMnfjWnk8kjfymFlaz')
temperatureloads = create_loads('https://drive.google.com/open?id=1jbZ3NkCXwP_gjlFYF7Fj-54Ua8SjGzR6')
ntuloads = create_loads('https://drive.google.com/open?id=1yyMNEzohhsWRMtP6FTz91Ytow6PVdVdC')


W0309 03:17:09.225687 140126429210496 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/googleapiclient/discovery_cache/__init__.py", line 41, in autodetect
    from . import file_cache
  File "/usr/local/lib/python2.7/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    'file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth')
ImportError: file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth


('Loading: ', '1-6mD_PTOYuVTPEtTK9pZUK4EUymj7K6V')
('Loading: ', '1sp1WCu_aavxN27mxB6LIv5qzm694VUJe')
('Loading: ', '1pMuQPazmrlovFSuMnfjWnk8kjfymFlaz')
('Loading: ', '1jbZ3NkCXwP_gjlFYF7Fj-54Ua8SjGzR6')
('Loading: ', '1yyMNEzohhsWRMtP6FTz91Ytow6PVdVdC')


In [ ]:
phloads

In [ ]:
# Cleanup Data Frames
# Convert columns which have interesting data to a numeric field (float) 
# Drop rows which do not have a clean numeric value (NaN)
# If we have multiple readings from the station (StationID) across different 
#     let's take the mean of each of the values for the StationID, thereby making 
#     StationID a unique key in the table
# TODO: Take mean only for same Month and retain the Month of the Year as part 
#       of the Unique Key - since quantities such as Temperature vary with the
#       Month of the Year
# Drop all other columns, while retaining the columns which are interesting to us

# Input parameters:
#    df: The data frame read from the Water Quality CSV file
#    col: The list of columns which are interesting for that data frame;
#         rest of the columns will be dropped 
def clean_my_df(df, col):
  # Convert the column to numeric
  df[col] = pd.to_numeric(df[col], errors="coerce")
  # drop any of the rows with that columns which are NaN
  df.dropna(subset=[col, 'StationID'], inplace=True)
  # Take the mean of all values for a given station and make the stationid unique in the dataframe
  df.groupby('StationID', as_index=False)[col].mean()
  #Drop all columns but StationID and col
  return df.filter(['StationID', col])

In [ ]:
phloads = clean_my_df(phloads, 'pH')
doloads = clean_my_df(doloads, 'DOmgL')
#TODO: temperature: convert from F to all C - For now we just ignored those rows
temperatureloads = clean_my_df(temperatureloads, 'TemperatureC')
ntuloads = clean_my_df(ntuloads, 'TurbidityNTU')

mergedf = pd.merge(phloads,temperatureloads,on='StationID', how='outer')
mergewithdodf = pd.merge(mergedf, doloads,on='StationID', how='outer')
mergedf = pd.merge(mergewithdodf,ntuloads,on='StationID', how='outer')


In [ ]:
# Let's pick temp + pH --> DO 
# Does temperature and pH have a correlation on DO

#Target DO: hence drop all unknowns from the pH and Temperature columns
#so that we can feed the clean data as input to the Keras model

'''
def prepare_ph_temp_to_do(mergedf):
  #Make a copy of the original mergedf
  dtpdf = mergedf
  # Clean out any rows which are not a number
  dtpdf.dropna(subset=['pH', 'TemperatureC', 'DOmgL'], inplace=True)

  # Split the dataframe into output (target) and input (features)
  # In this case DO is the output and Temp and pH are the inputs
  target_do_df = dtpdf[['DOmgL']]
  ph_temp_df = dtpdf[['TemperatureC', 'pH']]
  return ph_temp_df, target_do_df
'''


# Function which takes a df, input_cols list and output_cols list
# and produced two different dataframes - one with input_cols and other
# with output_cols
# This function drops any rows in the original mergedf with a NaN
def input_output_df(mergedf, input_cols, output_cols):
  #Make a copy of the original mergedf
  iodf = mergedf
  # Clean out any rows which are not a number
  all_cols = input_cols + output_cols
  iodf.dropna(subset=all_cols, inplace=True)

  # Split the dataframe into output (target) and input (features)
  # In this case DO is the output and Temp and pH are the inputs
  output_df = iodf[output_cols]
  input_df = iodf[input_cols]
  return input_df, output_df

'''
def prepare_ph_do_to_ntu(mergedf):
  #Make a copy of the original mergedf
  dtpdf = mergedf
  # Clean out any rows which are not a number
  dtpdf.dropna(subset=['pH', 'TurbidityNTU', 'DOmgL'], inplace=True)

  # Split the dataframe into output (target) and input (features)
  # In this case DO is the output and Temp and pH are the inputs
  output_df = dtpdf[['TurbidityNTU']]
  input_df = dtpdf[['DOmgL', 'pH']]
  return input_df, output_df
'''


'''
#input_pd_df, output_ntu_df = prepare_ph_do_to_ntu(mergedf)
input_pt_df, output_do_df = input_output_df(mergedf, ['DOmgL', 'pH'], ['TurbidityNTU'])
print ("pH + DO --> NTU")
print(output_ntu_df.head(3), input_pd_df.head(3))
'''

'''
# Convert the training data frame into a tensorflow dataset

#We did not need to convert the dataframe into a tensor dataset since Keras just used 
#the numpy arrays instead

def df_to_dataset(training_df, features, target):
  training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(training_df[features].values, tf.float32),
            tf.cast(training_df[target].values, tf.float32)
        )
    )
  )
  
  #for features_tensor, target_tensor in training_dataset:
  #  print('features:{} target:{}'.format(features_tensor, target_tensor))
  return training_dataset

features = ['pH', 'TemperatureC']
target='DOmgL'
train_ds = df_to_dataset(training_df, features, target)
'''

"\n# Convert the training data frame into a tensorflow dataset\n\n#We did not need to convert the dataframe into a tensor dataset since Keras just used \n#the numpy arrays instead\n\ndef df_to_dataset(training_df, features, target):\n  training_dataset = (\n    tf.data.Dataset.from_tensor_slices(\n        (\n            tf.cast(training_df[features].values, tf.float32),\n            tf.cast(training_df[target].values, tf.float32)\n        )\n    )\n  )\n  \n  #for features_tensor, target_tensor in training_dataset:\n  #  print('features:{} target:{}'.format(features_tensor, target_tensor))\n  return training_dataset\n\nfeatures = ['pH', 'TemperatureC']\ntarget='DOmgL'\ntrain_ds = df_to_dataset(training_df, features, target)\n"

In [ ]:
# THE TRAINING!!!!!!!
def train_my_data(input_df, output_df, num_neurons=10, loss_fn='mean_squared_error', optimizer='adam'):
  #create model from the input_df
  model = tf.keras.Sequential()

  #get number of columns in training data; input_df is the input training set dataframe
  n_cols = input_df.shape[1]
  n_output_cols = output_df.shape[1]

  #add model layers
  model.add(layers.Dense(num_neurons, activation='relu', input_shape=(n_cols,)))
  model.add(layers.Dense(10, activation='relu'))
  model.add(layers.Dense(n_output_cols))

  #compile model using mse as a measure of model performance
  model.compile(optimizer=optimizer, loss=loss_fn)
  #https://keras.io/losses/


  #set early stopping monitor so the model stops training when it won't improve anymore
  early_stopping_monitor = EarlyStopping(patience=3)

  # We first need to convert the dataframe into numpy array since tensor flow does not understand df
  # Also we need to make sure that the input and output float types match and so we will typecast it to float32
  input_array = input_df.to_numpy().astype('float32')
  output_array = output_df.to_numpy().astype('float32')

  # Yay !! Run the training now; adjust validation_split and epochs to see how the loss chages
  model.fit(input_array, output_array, validation_split=0.2, epochs=60, callbacks=[early_stopping_monitor])
  return model


#input_pt_df, output_do_df = prepare_ph_temp_to_do(mergedf)
input_pt_df, output_do_df = input_output_df(mergedf, ['TemperatureC', 'pH'], ['DOmgL'])
print('Train pH + Temp --> DO')
print(len(input_pt_df), len(output_do_df))
#model = train_my_data(input_pt_df, output_do_df, num_neurons=32, loss_fn='mean_absolute_error')
model = train_my_data(input_pt_df, output_do_df, num_neurons=32, optimizer='nadam', loss_fn='mean_absolute_error')


# Predict based on the model and input_df
# Print output_df side by side predictions
def please_predict(model, input_df, output_df):
  predictions =  model.predict(input_df)
  print(len(predictions), len(output_df))
  for a,b in zip(output_df.to_numpy(), predictions):
    print(a , b)

please_predict(model, input_pt_df, output_do_df)


Train pH + Temp --> DO
(378, 378)


AttributeError: ignored

In [ ]:
inrmloads.head(3)

In [ ]:
# From inrmloads, separate out the constituent water quality parameters into its own data frame and  
# merge them by station id
def create_param_df(param):
  df = inrmloads[inrmloads.CONSTIT == param]
  df = df[['SITE_QW_ID', 'WY','MONTH','TONS']]
  # TODO -  fixx up weird Water year calculation
  df.columns = ['StationID', 'Year', 'Month', param]
  return df

nh3df = create_param_df('NH3')
no3no2df = create_param_df('NO3_NO2')
opdf = create_param_df('OP')
sidf = create_param_df('SI')
tkndf = create_param_df('TKN')
tndf = create_param_df('TN')
tpdf = create_param_df('TP')

data_frames = [nh3df, no3no2df, opdf, sidf, tkndf, tndf, tpdf]
combinedf = reduce(lambda  left,right: pd.merge(left,right,on=['StationID', 'Year', 'Month'],
                                            how='outer'), data_frames)


input_df, output_df = input_output_df(combinedf, ['TKN'], ['TN'])
print ("TKN + TP --> TN", len(output_df))
print(output_df.head(3), input_df.head(3))
train_my_data(input_df, output_df, num_neurons=32, loss_fn='mean_squared_logarithmic_error')


('TKN + TP --> TN', 51678)
(          TN
192  28200.0
193  27200.0
194  31700.0,          TKN
192  11600.0
193  11700.0
194  14300.0)


AttributeError: ignored

In [ ]:
combinedf.head()

In [ ]:
input_df, output_df = input_output_df(combinedf, ['NH3', 'OP', 'TKN'], ['TP'])
print('Train OP + SI --> TP')
print(len(input_df), len(output_df))
#model = train_my_data(input_pt_df, output_do_df, num_neurons=32, loss_fn='mean_absolute_error')
model = train_my_data(input_df, output_df, num_neurons=10, optimizer='rmsprop', loss_fn='mean_squared_error')


Train OP + SI --> TP
(43759, 43759)


AttributeError: ignored

In [ ]:
# WE MAY NO LONGER NEED BELOW THIS
#inrmloads = pd.read_csv('mloads.csv', comment='#')
aggdf = pd.pivot_table(inrmloads,index=['SITE_ABB'],columns=['CONSTIT'],values=['TONS'],aggfunc=np.mean)
aggdf.columns=['NH3', 'NO3_NO2', 'OP', 'SI', 'TKN', 'TN', 'TP']
aggdf.head(30)
#organize data by constituent type (isolate only constituents, sort by constituents)
#default aggfunc is mean, but it is defined here


In [ ]:
aggdf
for col in aggdf.columns: 
    print(col) 
aggdf.plot.scatter(x=0,y=1)
aggdf.plot.scatter(x=('TONS', 'TN'),y=('TONS', 'TP'))
for x in range(0,6): #picking x-coordinates (from 7 column names)
  for y in range(0,6): #picking y-coordinates, plot all correlations
    if(x == y): #as long as x and and y are different column indices, plot the two values 
      continue 
    else:
      aggdf.plot.scatter(x,y)
      
      
#now: line of best fit, regression model

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

In [ ]:
graph = 
for x in range(0,6): #picking x-coordinates (from 7 column names)
  for y in range(0,6): #picking y-coordinates, plot all correlations
    for z in range(0,6):
     if(x == y == z): #as long as x and and y are different column indices, plot the two values 
      continue 
    else:
      ax.plot3D(x,y,z)
# Data for a three-dimensional line
# Data for three-dimensional scattered points


In [ ]:
aggdf = pd.pivot_table(inrmloads,index=['SITE_ABB'],columns =['CONSTIT'],values=['TONS'],aggfunc=np.mean)
aggdf

In [ ]:
for col in aggdf.columns: 
    print(col) 

aggdf.rename(columns={('TONS','NH3'): 'NH3'}, inplace=True)
aggdf.head(3)


In [ ]:
def calculate_NH3N(nh3):
  return nh3*0.82249559672

aggdf['NH3N'] = aggdf[('TONS', 'NH3')].apply(calculate_NH3N)
aggdf.head(3)

pH:   https://drive.google.com/open?id=1xNZhBcAEB8hk3ofZNwwUbzdm_s5ZelWt

FNU:https://drive.google.com/open?id=1PW-ZSwVQKbtTId_AtkzgZnS2MwNTL9SP

DO:   https://drive.google.com/open?id=1KVt0nkbSKqy4O7DbRxxRfRqpEMDfzeFN

Temp: https://drive.google.com/open?id=1RFGG2Xo6a2MYkJLVO7yAPpmqxtPjK5Pv
